In [2]:
!pip install kmodes

In [4]:
from kmodes import kprototypes
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import os

In [ ]:
df = pd.read_csv('')

# 1. 좌석 클러스터링

In [ ]:
def which_genre(genre, data):
  target = data[data['장르'] == genre].dropna(subset=['seat_grade'])
  df = target[['preorderandcancel', 'x_abs', 'y_abs', 'z', 'dist', '좌석가격', 'seat_grade']]
  input_data = target.copy()

  #obtain array of values
  data_array=df.values
  #specify data types
  data_array[:, 0:1] = data_array[:, 0:1].astype(str)
  data_array[:, 1:] = data_array[:, 1::].astype(float)

  return target, df, input_data, data_array

In [ ]:
genre_list = ['합창', '가족극', '기타', '독주', '복합장르', '성악', '실내악', '오페라', '재즈', '콘서트', '크로스오버']
target, df, input_data, data_array = which_genre('재즈' , df)

#create untrained model
untrained_model = kprototypes.KPrototypes(n_clusters=5,max_iter=10)

#predict clusters
clusters = untrained_model.fit_predict(data_array, categorical=[0])
input_data["Cluster labels"]=clusters
print("The clustered data is:")
input_data

In [ ]:
for i in range(len(genre_list)):

  print(genre_list[i], "클러스터링 시작")
  target, df, input_data, data_array = which_genre(genre_list[i] , df)

  #create untrained model
  untrained_model = kprototypes.KPrototypes(n_clusters=5,max_iter=10)

  #predict clusters
  clusters = untrained_model.fit_predict(data_array, categorical=[0])
  input_data["Cluster labels"]=clusters
  print("The clustered data is:")
  input_data

  input_data.to_csv("/content/drive/MyDrive/유나다 코드 정리/빅콘테스트/1. Data/가격 모델을 위한 클러스터별 파일/cluster_"+ genre_list[i] + ".csv")

  c0 = input_data[input_data['Cluster labels'] == 0]
  c1 = input_data[input_data['Cluster labels'] == 1]
  c2 = input_data[input_data['Cluster labels'] == 2]
  c3 = input_data[input_data['Cluster labels'] == 3]
  c4 = input_data[input_data['Cluster labels'] == 4]

  fig = plt.figure(figsize=(18, 10))
  ax = fig.add_subplot(111, projection='3d')

  print(genre_list[i], "클러스터링 시각화")

  ax.scatter(c0['x'], c0['y'], c0['z'], color = 'red', alpha = 0.1)
  ax.scatter(c1['x'], c1['y'], c1['z'], color = 'orange', alpha = 0.3)
  ax.scatter(c2['x'], c2['y'], c2['z'], color = 'blue', alpha = 0.05)
  ax.scatter(c3['x'], c3['y'], c3['z'], color = 'green', alpha = 0.1)
  ax.scatter(c4['x'], c4['y'], c4['z'], color = 'pink', alpha = 0.3)

  print("클러스터별 순위 평균")
  print("c0 :",c0['좌석가격'].mean()) #빨
  print("c1 :",c1['좌석가격'].mean()) #노
  print("c2 :",c2['좌석가격'].mean()) #파
  print("c3 :",c3['좌석가격'].mean()) #초
  print("c4 :",c4['좌석가격'].mean()) #핑

# 2. 가격 모델

In [ ]:
file_path= '/content/drive/MyDrive/유나다 코드 정리/빅콘테스트/1. Data/가격 모델을 위한 클러스터별 파일'
df_list = os.listdir(file_path)
df_list = sorted(df_list)

seat_dict = {'1층 A블록':210, '1층 B블록':260, '1층 C블록':286, '1층 D블록':260, '1층 E블록':210, '2층 A블록':127, '2층 BOX1':6, '2층 BOX2':8, '2층 BOX3':10, '2층 BOX4':10, '2층 BOX5':8, '2층 BOX6':6, '2층 B블록':84, '2층 C블록':82, '2층 D블록':84, '2층 E블록':127,
 '3층 A블록':56, '3층 BOX10':12, '3층 BOX11':9, '3층 BOX12':6, '3층 BOX7':6, '3층 BOX8':9, '3층 BOX9':12, '3층 B블록':31, '3층 C블록':46, '3층 D블록':49, '3층 E블록':46,'3층 F블록':31, '3층 G블록':56, '3층 M블록':30, '3층 N블록':30, '합창석 F블록':72, '합창석 G블록':130, '합창석 H블록':72}

In [ ]:
def ticket_price(df,dict,price):
  seat_list = df['층_블록'].unique()
  rank_list =  []
  for seat in seat_list:
    rank_list.append(df.loc[df['층_블록']==seat,'좌석가격'].values[0])
  calcul_df = pd.DataFrame({'층_블록':seat_list,'좌석가격':rank_list})
  calcul_df['좌석가격_비율'] = calcul_df['좌석가격'].replace(dict)
  calcul_df['좌석개수'] = calcul_df['층_블록'].replace(seat_dict)
  가격 =sum( calcul_df['좌석개수'] *calcul_df['좌석가격_비율'])
  a = price/가격
  calcul_df['실내악_최종_좌석_가격'] = calcul_df['좌석가격_비율']*a
  calcul_df['예메오픈가격'] = calcul_df['최종_좌석_가격']*0.7
  calcul_df['공연_1달전_가격'] = calcul_df['최종_좌석_가격']
  calcul_df['공연_15일전_가격'] = calcul_df['최종_좌석_가격']*1.15
  calcul_df['공연_6일전_가격'] = calcul_df['최종_좌석_가격']*1.3
  return calcul_df


In [ ]:
############### 가족극 #####################
cluster_가족극_df =  pd.read_csv(file_path +'/'+df_list[0],index_col=0)
가족극_ratio_dict = {0: 1.0, 1: 1.125, 2: 1.453, 3: 1.5, 4: 1.562, 5: 2.0, 6: 3.125}
가족극_df = ticket_price(cluster_가족극_df,가족극_ratio_dict,100000000)

############### 교향곡 #####################
cluster_교향곡_df =  pd.read_csv(file_path +'/'+df_list[1],index_col=0)
교향곡_ratio_dict= {0: 1.2613492063492062, 1: 1.2794021164021163, 2: 1.4664880382775118, 3: 1.565111650485437, 4: 1.5833908045977012, 5: 1.6507552083333337, 6: 1.6654411764705883, 7: 1.754929648241206,
 8: 1.787592233009709, 9: 1.9205936073059362, 10: 1.9509913793103448, 11: 1.9822925764192139, 12: 2.0920169491525424, 13: 2.0962944444444442, 14: 2.1032732558139537, 15: 2.1149326424870467,
 16: 2.1177298850574715, 17: 2.171054945054945, 18: 2.9809650349650347, 19: 3.053958041958042, 20: 3.0837157894736844, 21: 3.1405913978494624, 22: 3.490276073619632, 23: 3.49953125,
 24: 3.5276489361702126, 25: 3.5603205128205135, 26: 3.634464646464646, 27: 3.6625212765957444, 28: 4.392818518518519, 29: 4.439421875, 30: 4.599848148148149, 31: 5.294723183391004,
 32: 5.464431972789116, 33: 5.549827242524917}
교향곡_df = ticket_price(cluster_교향곡_df,교향곡_ratio_dict,100000000)

############### 기타 #####################
cluster_기타_df =  pd.read_csv(file_path +'/'+df_list[2],index_col=0)
기타_ratio_dict = {0: 1.0, 1: 2.473, 2: 3.261, 3: 3.78, 4: 3.904, 5: 5.249}
기타_df = ticket_price(cluster_기타_df,기타_ratio_dict,100000000)

############### 독주 #####################
cluster_독주_df =  pd.read_csv(file_path +'/'+df_list[3],index_col=0)
독주_ratiodict= {0: 1.14, 1: 1.1545599999999998, 2: 1.1697777777777778, 3: 1.1915555555555555, 4: 1.216448275862069, 5: 1.2264814814814815, 6: 1.2912962962962964, 7: 1.3027586206896553,
 8: 1.3407777777777778, 9: 1.3495333333333333, 10: 1.4268928571428574, 11: 1.4635757575757575, 12: 1.5612142857142857, 13: 1.6230645161290322, 14: 1.6514193548387097, 15: 1.6743333333333332,
 16: 1.6879032258064517, 17: 1.6968064516129033, 18: 1.8720400000000001, 19: 1.9070799999999999, 20: 1.9545185185185188, 21: 1.9603243243243242, 22: 2.071, 23: 2.0765000000000002,
 24: 2.2164814814814813, 25: 2.466270270270271, 26: 2.468810810810811, 27: 2.803588235294118, 28: 3.057342105263158, 29: 3.107, 30: 3.1392162162162167, 31: 3.324263157894737,
 32: 3.5527894736842107, 33: 3.5592162162162153}
독주_df = ticket_price(cluster_독주_df,독주_ratiodict,100000000)

############### 복합장르 #####################
cluster_복합장르_df =  pd.read_csv(file_path +'/'+df_list[4],index_col=0)
복합장르_ratio_dict = {0: 1.013, 1: 1.02, 2: 1.0434999999999999, 3: 1.1095000000000002, 4: 1.206, 5: 1.2345, 6: 1.2584, 7: 1.2870000000000001, 8: 1.3026, 9: 1.3072000000000001,
 10: 1.327, 11: 1.3416000000000001, 12: 1.342, 13: 1.35325, 14: 1.3675000000000002, 15: 1.3967500000000002, 16: 1.58975, 17: 1.7804000000000002,
 18: 1.818, 19: 1.9746000000000001, 20: 2.0524, 21: 2.054, 22: 2.0672, 23: 2.0957999999999997, 24: 2.118, 25: 2.1475,
 26: 2.1580000000000004, 27: 2.1761999999999997, 28: 2.5638, 29: 2.6209999999999996, 30: 2.6532000000000004, 31: 3.2388000000000003, 32: 3.2481999999999998, 33: 3.2826}
복합장르_df = ticket_price(cluster_복합장르_df,복합장르_ratio_dict,100000000)

############### 성악 #####################
cluster_성악_df = pd.read_csv('/content/drive/MyDrive/clustering/'+df_list[5],index_col=0)
성악_ratio_dict = {0: 1.1186923076923077, 1: 1.381375,
 2: 1.39825, 3: 1.401333333333333, 4: 1.4017333333333333, 5: 1.419375, 6: 1.4756923076923074, 7: 1.5227499999999998, 8: 1.5894545454545457,
 9: 1.5976428571428571, 10: 1.6191428571428568, 11: 1.6366923076923074, 12: 1.6633000000000002, 13: 1.6968, 14: 1.812315789473684, 15: 1.8141111111111112, 16: 1.8243333333333336,
 17: 1.90915, 18: 2.329692307692308, 19: 2.3618148148148146, 20: 2.3920000000000003, 21: 2.5316875, 22: 2.5675624999999997, 23: 2.5850666666666666, 24: 2.671333333333334,
 25: 2.767809523809524, 26: 3.075454545454545, 27: 3.189321428571429, 28: 3.2414166666666664, 29: 3.289571428571428, 30: 3.443739130434783, 31: 4.395, 32: 4.462714285714286, 33: 4.538538461538462}
성악_df = ticket_price(cluster_성악_df,성악_ratio_dict,100000000)

############### 실내악 #####################
cluster_실내악_df = pd.read_csv(file_path +'/'+df_list[6],index_col=0)
실내악_ratio_dict = {0: 1.204, 1: 1.2075, 2: 1.214, 3: 1.2412727272727275, 4: 1.2455000000000003, 5: 1.2685, 6: 1.2795999999999998, 7: 1.3494285714285716, 8: 1.3773333333333333, 9: 1.3982222222222223,
 10: 1.5322000000000002, 11: 1.5507000000000002,12: 1.6444545454545456, 13: 1.700090909090909, 14: 1.9154166666666665, 15: 1.9283571428571429, 16: 2.1715833333333334, 17: 2.2427333333333332,
 18: 2.306416666666667, 19: 2.3184285714285715, 20: 2.389347826086957, 21: 2.4424347826086956, 22: 2.4426153846153844, 23: 2.484, 24: 2.4970833333333333, 25: 2.7489999999999997,
 26: 2.8152916666666665, 27: 2.919791666666667, 28: 3.5502380952380954, 29: 3.5801428571428575, 30: 3.8882105263157896, 31: 4.23164, 32: 4.3009130434782605, 33: 4.416239999999999}
실내악_df = ticket_price(cluster_실내악_df,실내악_ratio_dict,100000000)

############### 오페라 #####################
cluster_오페라_df = pd.read_csv(file_path +'/'+df_list[7],index_col=0)
오페라_ratio_dict ={0: 1.2, 1: 1.3846666666666667, 2: 1.4631666666666667, 3: 1.6138333333333332, 4: 1.7364000000000002, 5: 1.7456, 6: 1.848, 7: 2.045571428571429,
 8: 2.063833333333333, 9: 2.114142857142857, 10: 2.12425, 11: 2.131, 12: 2.1486666666666667, 13: 2.1725000000000003, 14: 2.1888571428571426, 15: 2.3333333333333335,
 16: 2.6582500000000002, 17: 2.73575, 18: 2.97, 19: 3.1455, 20: 3.2284285714285708, 21: 3.2615000000000003, 22: 3.594857142857143, 23: 3.6750000000000003,
 24: 3.6918333333333333, 25: 3.798875, 26: 3.807857142857143, 27: 3.945888888888889, 28: 3.956285714285715, 29: 5.4472499999999995, 30: 5.730555555555556, 31: 5.848222222222223, 32: 6.157875}
오페라_df = ticket_price(cluster_오페라_df,오페라_ratio_dict,100000000)

############### 재즈 #####################
cluster_재즈_df =pd.read_csv(file_path +'/'+df_list[8],index_col=0)
재즈_ratio_dict = {0: 1.0, 1: 1.145, 2: 1.145, 3: 1.186, 4: 1.244, 5: 1.249, 6: 1.294, 7: 1.364, 8: 1.364, 9: 1.391, 10: 1.391, 11: 1.411, 12: 1.411, 13: 1.424, 14: 1.439, 15: 1.473,
 16: 1.496, 17: 1.541, 18: 1.823, 19: 1.836, 20: 1.845, 21: 1.846, 22: 1.872, 23: 1.964}
재즈_df = ticket_price(cluster_재즈_df,재즈_ratio_dict,100000000)

############### 콘서트 #####################
cluster_콘서트_df = pd.read_csv(file_path +'/'+df_list[9],index_col=0)
콘서트_ratio_dict = {0: 1.0615, 1: 1.4191666666666667, 2: 1.563, 3: 1.6177142857142859, 4: 1.7373333333333332, 5: 1.87525, 6: 1.8938571428571431, 7: 1.907875, 8: 2.1315, 9: 2.2199999999999998, 10: 2.2619999999999996, 11: 2.471625, 12: 2.538222222222222, 13: 2.5540909090909087,
 14: 2.674375, 15: 2.7326249999999996, 16: 2.7477272727272726, 17: 2.869555555555556, 18: 2.8705, 19: 3.2518888888888884, 20: 3.45475, 21: 3.678777777777778, 22: 3.755777777777778,
 23: 3.907666666666666, 24: 4.018083333333334, 25: 4.113666666666667, 26: 4.452, 27: 4.789, 28: 4.819384615384616, 29: 5.067928571428571, 30: 5.381857142857143,
 31: 5.709428571428572, 32: 6.3310666666666675, 33: 6.705642857142857}
콘서트_df = ticket_price(cluster_콘서트_df,콘서트_ratio_dict,100000000)

############### 크로스오버 #####################
cluster_크로스오버_df = pd.read_csv('/content/drive/MyDrive/clustering/'+df_list[10],index_col=0)
크로스오버_ratio_dict = {0: 1.0, 1: 1.0, 2: 1.016, 3: 1.0739999999999998, 4: 1.077, 5: 1.077, 6: 1.077, 7: 1.077, 8: 1.077, 9: 1.0815000000000001, 10: 1.097, 11: 1.1099999999999999, 12: 1.436, 13: 1.436, 14: 1.436, 15: 1.575,
 16: 1.6035, 17: 1.795, 18: 1.795, 19: 1.795, 20: 1.8119999999999998, 21: 2.154, 22: 2.263, 23: 2.3975, 24: 2.6975, 25: 2.877}
크로스오버_df = ticket_price(cluster_크로스오버_df,크로스오버_ratio_dict,100000000)

############### 클래식 #####################
cluster_클래식_df = pd.read_csv('/content/drive/MyDrive/clustering/'+df_list[11],index_col=0)
클래식_ratio_dict = {0: 1.356596153846154, 1: 1.3610675675675674, 2: 1.4199883720930233, 3: 1.442576923076923, 4: 1.486, 5: 1.5152317073170731, 6: 1.5221724137931034, 7: 1.5229166666666667,
 8: 1.548258823529412, 9: 1.5740185185185183, 10: 1.5794050632911392, 11: 1.689639534883721, 12: 2.0341688311688313, 13: 2.0354941176470587, 14: 2.0503417721518984, 15: 2.357901960784314,
 16: 2.376058823529412, 17: 2.4342522522522523, 18: 2.4713852459016397, 19: 2.585505617977528, 20: 3.0344886363636365, 21: 3.0399425287356325, 22: 3.0456742424242425, 23: 3.069818965517242,
 24: 3.0755846153846154, 25: 3.105730337078652, 26: 3.130075, 27: 3.1337108433734935, 28: 3.146798076923077, 29: 3.2714175824175817, 30: 3.5372148760330577, 31: 4.044353846153846, 32: 4.077064, 33: 4.356160305343511}
클래식_df = ticket_price(cluster_클래식_df,클래식_ratio_dict,100000000)

############### 합창 #####################
cluster_합창_df = pd.read_csv('/content/drive/MyDrive/clustering/'+df_list[12],index_col=0)
합창_ratio_dict = {0: 1.5036206896551725, 1: 1.5686296296296294, 2: 1.5813333333333333, 3: 1.5987575757575758, 4: 1.692071428571429, 5: 1.7499565217391306, 6: 1.8388124999999997, 7: 1.8615909090909093,
 8: 1.8619545454545452, 9: 1.9336129032258065, 10: 1.9745714285714286, 11: 1.9983999999999997, 12: 2.0707142857142857, 13: 2.0744, 14: 2.1006153846153848, 15: 2.148,
 16: 2.1634615384615383, 17: 2.3564166666666666, 18: 2.394473684210526, 19: 2.739909090909091, 20: 2.792904761904762, 21: 2.892452380952381, 22: 3.028311111111111, 23: 3.0706382978723403,
 24: 3.2616, 25: 3.485024390243902, 26: 3.4973, 27: 3.5544, 28: 3.6021111111111113, 29: 4.238300000000001, 30: 4.829235294117647, 31: 4.9433750000000005, 32: 5.135700000000001}
합창_df = ticket_price(cluster_합창_df,합창_ratio_dict,100000000)

In [ ]:
dfs = [합창_df, 가족극_df, 교향곡_df, 기타_df, 독주_df, 복합장르_df,
       실내악_df, 성악_df, 오페라_df, 재즈_df, 콘서트_df, 크로스오버_df, 클래식_df]

# 층_블록 열을 기준으로 외부 조인하여 price_df를 생성합니다.
price_df = 합창_df[['층_블록']]
for df in dfs:
    price_df = pd.merge(price_df, df[['층_블록', df.columns[-1]]], how='outer', on='층_블록')

In [ ]:
price_df = 합창_df[['층_블록']]
price_df = pd.merge(price_df,가족극_df[['층_블록', '가족극_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,교향곡_df[['층_블록','교향곡_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,기타_df[['층_블록','기타_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,독주_df[['층_블록','독주_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,복합장르_df[['층_블록','복합장르_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,실내악_df[['층_블록','실내악_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,성악_df[['층_블록','성악_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,오페라_df[['층_블록','오페라_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,재즈_df[['층_블록','재즈_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,콘서트_df[['층_블록','콘서트_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,크로스오버_df[['층_블록','크로스오버_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,클래식_df[['층_블록','클래식_최종_좌석_가격']], how='outer',on='층_블록')
price_df = pd.merge(price_df,합창_df[['층_블록','합창_최종_좌석_가격']], how='outer',on='층_블록')

In [ ]:
# dfs = [합창_df, 가족극_df, 교향곡_df, 기타_df, 독주_df, 복합장르_df,
      #  실내악_df, 성악_df, 오페라_df, 재즈_df, 콘서트_df, 크로스오버_df, 클래식_df]
# price_df = 합창_df[['층_블록']]
# for df in dfs:
    # price_df = pd.merge(price_df, df[['층_블록', df.columns[-1]]], how='outer', on='층_블록')
